In [2]:
import numpy as np
config = {
    # base info about dataset
    'train_num':44375,
    'test_num':21435,
    'test_num':21435,

    # paradim setting
    'sentence_len':8,
    'use_stopwords':True,

}
stoplist = set('a the this that these those of on in to for'.split())

In [3]:
import utils
import pandas as pd
import re
from copy import deepcopy

Q_set,A_set = utils.load_json_file(path = '../data',cleaned=False)
for item in Q_set:
    qst = item['question']
    qst = qst.lower()
    qst = re.sub(r"[^a-zA-Z]+", r" ",qst)
    qst = qst.strip()
    qst_vocab = qst.split(' ')
    qst_vocab = utils.sentence_align(qst_vocab,config['sentence_len'])
    item['question'] = deepcopy(qst_vocab)
    
print(len(Q_set))

Q_df = pd.DataFrame(Q_set)
A_df = pd.DataFrame(A_set)

Q_A_df = Q_df.merge(A_df,on = ['question_id','image_id'])
Q_A_df = Q_A_df[['question_id','image_id','question','multiple_choice_answer']]
Q_A_df['annotation'] = Q_A_df['multiple_choice_answer']
Q_A_df = Q_A_df[['question_id','image_id','question','annotation']]

print(Q_A_df.shape)
Q_A_df.head()


train : 44375, eval : 21435, test : 21435
87245
(87245, 4)


,question_id,image_id,question,annotation
0,393223003,393223,"[is, she, brushing, <INS>, <INS>, <INS>, <INS>...",yes
1,393227001,393227,"[what, is, this, man, riding, on, <INS>, <INS>]",skateboard
2,131074002,131074,"[are, the, walls, done, in, a, summery, color]",yes
3,393230003,393230,"[what, is, the, man, doing, <INS>, <INS>, <INS>]",skiing
4,393230005,393230,"[what, is, the, person, wearing, <INS>, <INS>,...",skis


In [ ]:
import word2vec
word2vec_model = word2vec.load_word2vec('./word_vec/embedding.txt')
def qst2tenser(qst,model):
    '''
    note: 尚未测试,可能有bug
    '''
    qst_tensor = []
    for item in qst:
        if item == '<INS>':
            qst_tensor.append([[0]*100])
        else :
            qst_tensor.append(model[item])
    return qst_tensor

rst = []
for item in  Q_A_df['question']:
    rst.append(qst2tenser(item,word2vec_model))

Q_A_df['question_tensor'] = rst
Q_A_df.head()
# word2vec_model